## 第一问与第二问的区别及其数学表达

### 1. 第一问：滞销情况下的浪费

在第一问的条件下，农作物的种植和销售受限于市场需求，所有超出市场需求的部分都会被认为是滞销，从而导致浪费。这意味着，超过预期销售量的产量部分将不会产生任何收入。

#### 数学表达

对于每种作物$j$和每个地块$i$，如果作物的产量超出市场销售量，那么超出部分无法销售。我们定义如下变量：

- $y_j$: 作物$j$的单位亩产量（斤/亩）
- $x_{i,j,y}$: 在地块$i$上种植作物$j$在年份$y$的种植面积（亩）
- $S_j$: 作物$j$的市场销售量（斤）
- $P_j$: 作物$j$的销售单价（元/斤）
- $C_{i,j}$: 作物$j$在地块$i$的种植成本（元/亩）

目标函数是最大化总利润（净收益），考虑到浪费的情况：

$$
\text{Maximize} \sum_{i,j} \left( P_j \cdot y_j \cdot x_{i,j,y} - C_{i,j} \cdot x_{i,j,y} \right)
$$

同时加入约束条件：

$$
y_j \cdot x_{i,j,y} \leq S_j, \quad \forall i, j, y
$$

该约束确保了每种作物的产量不会超过其市场销售量，防止出现无法销售的浪费情况。

### 2. 第二问：滞销情况下降价处理

在第二问的条件下，农作物的种植和销售同样受限于市场需求，但超出市场需求的部分不会被完全浪费掉，而是按原销售价格的50%降价出售。这样，即使超出部分的产量也能产生一定的收入，只是价格较低。

#### 数学表达

我们保留第一问中的变量定义，同时引入新的变量和参数：

- $P'_j$: 作物$j$超出部分的降价销售单价，定义为 $P'_j = 0.5 \times P_j$

目标函数在考虑降价销售的情况下修改为：

$$
\text{Maximize} \sum_{i,j} \left( P_j \cdot \min(y_j \cdot x_{i,j,y}, S_j) - C_{i,j} \cdot x_{i,j,y} + P'_j \cdot \max(0, y_j \cdot x_{i,j,y} - S_j) \right)
$$

这里的目标函数包含两部分：

1. **正常销售收入**：对于产量不超过市场需求的部分，按正常单价$P_j$销售。
2. **滞销降价收入**：对于超过市场需求的部分，按降价单价$P'_j$销售。

### 区别总结

1. **超出部分处理方式**：
   - **第一问**：超出部分无法销售，导致浪费。
   - **第二问**：超出部分按50%降价销售，仍能产生一定收入。

2. **目标函数表达式的不同**：
   - **第一问**：目标函数只考虑正常销售收入，浪费部分不产生任何收入。
   - **第二问**：目标函数不仅考虑正常销售收入，还考虑了滞销降价收入。

3. **约束条件的变化**：
   - **第一问**：有销售量约束，确保产量不超过市场需求。
   - **第二问**：放宽了对超出产量的处理，通过降价销售的方式产生额外收入。


In [1]:
import pulp
import pandas as pd
import numpy as np
from scipy.optimize import linprog
import random

RANDOM_SEED = 96
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# 文件路径
planting_data_file = './2023年的种植数据与销售量.xlsx'
crop_sales_file = './各作物聚合后销售量与价格.xlsx'
crop_land_file = './各作物适合种植的地块类型与季别.xlsx'
land_info_file = './乡村的现有耕地.xlsx'

# 加载表格
planting_data = pd.read_excel(planting_data_file)
crop_sales_data = pd.read_excel(crop_sales_file)
crop_land_data = pd.read_excel(crop_land_file)
land_info_data = pd.read_excel(land_info_file)

# 提取地块和作物信息
land_names = land_info_data['地块名称'].unique().tolist()

# 根据 crop_land_data 筛选适合第一季和第二季的作物
def get_suitable_crops(season):
    if season == '第一季':
        suitable_crops = crop_land_data[crop_land_data[f'水浇地{season}'] == 1]['作物名称'].tolist() + \
                         crop_land_data[crop_land_data[f'普通大棚{season}'] == 1]['作物名称'].tolist() + \
                         crop_land_data[crop_land_data[f'智慧大棚{season}'] == 1]['作物名称'].tolist() + \
                         crop_land_data[crop_land_data[f'平旱地'] == 1]['作物名称'].tolist() + \
                         crop_land_data[crop_land_data[f'梯田'] == 1]['作物名称'].tolist() + \
                         crop_land_data[crop_land_data[f'山坡地'] == 1]['作物名称'].tolist() + \
                         crop_land_data[crop_land_data[f'水浇地'] == 1]['作物名称'].tolist()
    else:
        suitable_crops = crop_land_data[crop_land_data[f'水浇地{season}'] == 1]['作物名称'].tolist() + \
                         crop_land_data[crop_land_data[f'普通大棚{season}'] == 1]['作物名称'].tolist() + \
                         crop_land_data[crop_land_data[f'智慧大棚{season}'] == 1]['作物名称'].tolist()
    
    return suitable_crops

suitable_crops_first_season = get_suitable_crops('第一季')
suitable_crops_second_season = get_suitable_crops('第二季')

# 过滤不在suitable_crops中的作物
crops_first_season = [crop for crop in planting_data['作物名称_x'].unique() if crop in suitable_crops_first_season]
crops_second_season = [crop for crop in planting_data['作物名称_x'].unique() if crop in suitable_crops_second_season]

# 第二季地块
land_names_second_season = ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 
                            'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 
                            'F1', 'F2', 'F3', 'F4']

# 随机选择作物的数量,减少计算量
num_crops_to_select = 9

# 为第一季和第二季创建地块与作物的映射
def create_land_crop_mapping(land_names, crops):
    land_crop_mapping = {}
    for land in land_names:
        if len(crops) > num_crops_to_select:
            selected_crops = random.sample(crops, num_crops_to_select)
        else:
            selected_crops = crops  # 如果作物少于9种，全部选中
        land_crop_mapping[land] = selected_crops
    return land_crop_mapping

# 第一季
land_crop_mapping_first_season = create_land_crop_mapping(land_names, crops_first_season)

# 第二季
land_crop_mapping_second_season = create_land_crop_mapping(land_names_second_season, crops_second_season)

# 定义优化函数
def optimize_land_crop(land_crop_mapping, crops, season, year):
    # 初始化决策变量和目标函数
    decision_vars = {}
    objective_coeffs = []
    
    for land, selected_crops in land_crop_mapping.items():
        for crop in selected_crops:
            decision_vars[(crop, land)] = 0

            land_type = land_info_data[land_info_data['地块名称'] == land]['地块类型'].values[0]
            yield_data = planting_data[(planting_data['作物名称_x'] == crop) & (planting_data['地块类型'] == land_type)]['亩产量/斤'].values
            price_data = crop_sales_data[crop_sales_data['作物名称'] == crop]['销售单价/(元/斤)'].values
            cost_data = planting_data[(planting_data['作物名称_x'] == crop) & (planting_data['地块类型'] == land_type)]['种植成本/(元/亩)'].values
            sale_2023 = crop_sales_data[crop_sales_data['作物名称'] == crop]['销售量/斤'].values
            if len(sale_2023) > 0:
                sale_2023 = sale_2023[0]
            else:
                sale_2023 = 0

            if len(yield_data) > 0 and len(price_data) > 0 and len(cost_data) > 0:
                yield_per_acre = yield_data[0]
                price_per_unit = price_data[0]
                cost_per_acre = cost_data[0]
                reduced_price = price_per_unit * 0.5
                net_revenue = (yield_per_acre * price_per_unit - cost_per_acre)
                reduced_revenue = max(0, (yield_per_acre - sale_2023) * reduced_price)  # 滞销收入
                objective_coeffs.append(net_revenue + reduced_revenue)  # 目标函数：总净收入加上滞销收入
            else:
                objective_coeffs.append(0)

    objective_coeffs = np.array(objective_coeffs) * -1

    # 重新构建约束条件
    A_ub = []
    b_ub = []

    # 约束条件1：总面积约束
    for land in land_crop_mapping.keys():
        constraint = np.zeros(len(decision_vars))
        for i, (crop, land_name) in enumerate(decision_vars.keys()):
            if land_name == land:
                constraint[i] = 1
        A_ub.append(constraint)
        b_ub.append(land_info_data[land_info_data['地块名称'] == land]['地块面积/亩'].values[0])

    # 约束条件2：最小种植面积约束
    for land in land_crop_mapping.keys():
        min_area = 0.1 * land_info_data[land_info_data['地块名称'] == land]['地块面积/亩'].values[0]
        for crop in land_crop_mapping[land]:
            constraint = np.zeros(len(decision_vars))
            for i, (crop_name, land_name) in enumerate(decision_vars.keys()):
                if crop_name == crop and land_name == land:
                    constraint[i] = -1
            A_ub.append(constraint)
            b_ub.append(-min_area)

    # 约束条件3：三年内至少种植一次豆类作物
    beans_crops = crop_land_data[crop_land_data['作物类型'].str.contains('粮食（豆类）')]['作物名称'].tolist()
    for land in land_crop_mapping.keys():
        constraint = np.zeros(len(decision_vars))
        for i, (crop, land_name) in enumerate(decision_vars.keys()):
            if land_name == land and crop in beans_crops:
                constraint[i] = -1
        A_ub.append(constraint)
        b_ub.append(0)  # 三年内种植豆类作物的约束（具体实施时可能需要调整）

    # 约束条件4：销售量约束
    apply_sales_constraint = False
    if apply_sales_constraint:
        for crop in crops:
            total_yield = sum([
                decision_vars[(crop, land)] * planting_data[
                    (planting_data['作物名称_x'] == crop) & 
                    (planting_data['地块类型'] == land_info_data[land_info_data['地块名称'] == land]['地块类型'].values[0])
                ]['亩产量/斤'].values[0]
                for land in land_crop_mapping.keys() if (crop, land) in decision_vars
            ])
            constraint = np.zeros(len(decision_vars))
            for i, (crop_name, land_name) in enumerate(decision_vars.keys()):
                if crop_name == crop:
                    constraint[i] = 1
            A_ub.append(constraint)
            b_ub.append(sale_2023)  # 销售量约束

    A_ub = np.array(A_ub)
    b_ub = np.array(b_ub)

    # 优化模型
    result = linprog(c=objective_coeffs, A_ub=A_ub, b_ub=b_ub, method='highs')

    # 构建优化结果的表格输出
    if result.success:
        optimal_areas = result.x
        solution = {}
        
        for i, (crop, land) in enumerate(decision_vars.keys()):
            if land not in solution:
                solution[land] = {}
            solution[land][crop] = optimal_areas[i]

        # 确保结果表格中包含所有作物
        all_crops = sorted(set(crop_sales_data['作物名称'].to_list()))  # 所有作物的集合（去重并排序）
        
        # 构建结果表格
        results = pd.DataFrame(columns=['年', '季别', '地块名'] + all_crops)
        
        for land, crop_areas in solution.items():
            season_data = {'年': year, '季别': season, '地块名': land}
            for crop in all_crops:
                season_data[crop] = crop_areas.get(crop, 0)  # 如果该作物不在该地块中，则面积为0
            
            results = results._append(season_data, ignore_index=True)

        return results
    else:
        print(f"{year}年{season}优化失败，无法生成结果表格。")
        return None

# 迭代计算 2024~2030 年的结果
years = list(range(2024, 2031))
all_results = []

for year in years:
    # 第一季
    results_first_season = optimize_land_crop(land_crop_mapping_first_season, crops_first_season, '第一季', year)
    # 第二季
    results_second_season = optimize_land_crop(land_crop_mapping_second_season, crops_second_season, '第二季', year)

    # 合并结果
    if results_first_season is not None and results_second_season is not None:
        combined_results = pd.concat([results_first_season, results_second_season], ignore_index=True)
        all_results.append(combined_results)

# 最后保存结果
if all_results:
    final_results = pd.concat(all_results, ignore_index=True)
    final_results.to_excel('2024至2030年农作物种植方案.xlsx', index=None)

In [2]:
# 将所有结果写入Excel文件
if all_results:
    final_results = pd.concat(all_results, ignore_index=True)
    final_results.to_excel('2024至2030年农作物种植方案.xlsx', index=None)

In [3]:
final_results

,年,季别,地块名,刀豆,包菜,南瓜,土豆,大白菜,大麦,小青菜,...,豇豆,辣椒,青椒,香菇,高粱,黄心菜,黄瓜,黄豆,黍子,黑豆
0,2024,第一季,A1,8.0,8.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2024,第一季,A2,5.5,0.0,11.0,0,0,0,0,...,0,5.5,0,0,5.5,0,0,5.5,0,0
2,2024,第一季,A3,0.0,0.0,0,3.5,0,0,3.5,...,0,0,3.5,0,7.0,0,3.5,0,0,0
3,2024,第一季,A4,7.2,0.0,0,0,0,7.2,7.2,...,0,7.2,7.2,0,0,7.2,0,0,0,0
4,2024,第一季,A5,6.8,0.0,0,0,0,0,6.8,...,0,0,0,0,0,0,0,0,0,6.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,2030,第二季,E16,0.06,0,0,0,0.0,0,0.06,...,0,0,0.06,0,0,0,0.06,0,0,0
570,2030,第二季,F1,0,0.06,0,0,0.0,0,0,...,0.06,0,0,0,0,0,0.12,0,0,0
571,2030,第二季,F2,0,0.06,0,0,0.0,0,0,...,0.06,0,0.06,0,0,0,0.12,0,0,0
572,2030,第二季,F3,0.06,0,0,0,0.06,0,0,...,0,0.06,0.06,0,0,0.06,0.0,0,0,0


In [4]:


# 定义新的列名
new_columns = [
    '季别', '地块名', '黄豆', '黑豆', '红豆', '绿豆', '爬豆', '小麦', '玉米', '谷子', '高粱', '黍子', '荞麦', '南瓜',
    '红薯', '莜麦', '大麦', '水稻', '豇豆', '刀豆', '芸豆', '土豆', '西红柿', '茄子', '菠菜', '青椒', '菜花', '包菜',
    '油麦菜', '小青菜', '黄瓜', '生菜', '辣椒', '空心菜', '黄心菜', '芹菜', '大白菜', '白萝卜', '红萝卜', '榆黄菇',
    '香菇', '白灵菇', '羊肚菌'
]

# 创建一个 Pandas Excel writer 对象
with pd.ExcelWriter('2024至2030年农作物种植方案Q1_2.xlsx', engine='xlsxwriter') as writer:
    # 遍历每个年份并将相应的数据写入到不同的工作表中
    for year in range(2024, 2031):
        # 选择对应年份的数据
        year_data = final_results[final_results['年'] == year]
        
        # 删除年份列
        year_data = year_data.drop(columns=['年'])
        
        # 确保列名顺序符合要求
        year_data = year_data.reindex(columns=new_columns, fill_value=0)
        
        # 将数据写入到 Excel 文件的对应工作表中
        year_data.to_excel(writer, sheet_name=str(year), index=False)

